In [1]:
from pathlib import Path
import pandas as pd

RAW_DIR = Path("../../data/raw")
PROCESSED_DIR = Path("../../data/processed")


## Customers

In [3]:
customers = pd.read_csv(RAW_DIR / "olist_customers_dataset.csv")

In [4]:
# Clean
c_cleaned=customers.drop_duplicates(subset=["customer_id"])
c_cleaned["customer_unique_id"] = customers["customer_unique_id"].str.strip()

In [5]:
# Save cleaned file
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
customers.to_csv(PROCESSED_DIR / "customers_clean.csv", index=False)

## Orders

In [6]:
orders = pd.read_csv(RAW_DIR / "olist_orders_dataset.csv")

In [7]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [8]:
date_cols = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]
for col in date_cols:
    orders[col] = pd.to_datetime(orders[col], errors='coerce')


In [9]:
orders['order_status'].unique()

array(['delivered', 'invoiced', 'shipped', 'processing', 'unavailable',
       'canceled', 'created', 'approved'], dtype=object)

In [10]:
orders=orders[~orders['order_status'].isin(['canceled','unavailable'])]

In [11]:
orders_rfm = orders.loc[orders['order_status'] == 'delivered', ['order_id', 'customer_id', 'order_purchase_timestamp']]
orders_delivery = orders[orders['order_status'].isin(['delivered','shipped','invoiced'])]


In [12]:
orders_rfm.info()

<class 'pandas.core.frame.DataFrame'>
Index: 96478 entries, 0 to 99440
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   order_id                  96478 non-null  object        
 1   customer_id               96478 non-null  object        
 2   order_purchase_timestamp  96478 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 2.9+ MB


In [13]:
orders_rfm.head()

,order_id,customer_id,order_purchase_timestamp
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,2018-07-24 20:41:37
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,2018-08-08 08:38:49
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,2017-11-18 19:28:06
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,2018-02-13 21:18:39


## Payments

In [14]:
payments = pd.read_csv(RAW_DIR/"olist_order_payments_dataset.csv")

In [15]:
payments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [16]:
payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


In [18]:
payments['order_id'].unique()

array(['b81ef226f3fe1789b1e8b2acac839d17',
       'a9810da82917af2d9aefd1278f1dcfa0',
       '25e8ea4e93396b6fa0d3dd708e76c1bd', ...,
       '32609bbb3dd69b3c066a6860554a77bf',
       'b8b61059626efa996a60be9bb9320e10',
       '28bbae6599b09d39ca406b747b6632b1'], shape=(99440,), dtype=object)

In [20]:
payments.isnull().sum()

order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64

In [21]:
payments.duplicated().sum()

np.int64(0)

In [24]:
payment_agg=payments.groupby('order_id',as_index=False)['payment_value'].sum()

In [25]:
payment_agg.head()

,order_id,payment_value
0,00010242fe8c5a6d1ba2dd792cb16214,72.19
1,00018f77f2f0320c557190d7a144bdd3,259.83
2,000229ec398224ef6ca0657da4fc703e,216.87
3,00024acbcdf0a6daa1e931b038114c75,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,218.04


In [26]:
payment_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99440 entries, 0 to 99439
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   order_id       99440 non-null  object 
 1   payment_value  99440 non-null  float64
dtypes: float64(1), object(1)
memory usage: 1.5+ MB


## Combining Orders and payments

In [27]:
orders_rfm=orders_rfm.merge(payment_agg,on='order_id',how='left')

In [28]:
orders_rfm.head()

,order_id,customer_id,order_purchase_timestamp,payment_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,38.71
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,2018-07-24 20:41:37,141.46
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,2018-08-08 08:38:49,179.12
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,2017-11-18 19:28:06,72.20
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,2018-02-13 21:18:39,28.62


## % of Customers using different payment methods

In [29]:
payments["payment_type"].value_counts(normalize=True) * 100


payment_type
credit_card    73.922376
boleto         19.043952
voucher         5.558978
debit_card      1.471806
not_defined     0.002888
Name: proportion, dtype: float64

In [30]:
orders_rfm.isnull().sum()

order_id                    0
customer_id                 0
order_purchase_timestamp    0
payment_value               1
dtype: int64

In [31]:
orders_rfm=orders_rfm.dropna(subset=['payment_value'])

In [32]:
orders_rfm.info()

<class 'pandas.core.frame.DataFrame'>
Index: 96477 entries, 0 to 96477
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   order_id                  96477 non-null  object        
 1   customer_id               96477 non-null  object        
 2   order_purchase_timestamp  96477 non-null  datetime64[ns]
 3   payment_value             96477 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 3.7+ MB


In [34]:
c_cleaned.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


## Merging Customers and orders_rfm

In [35]:
customers_clean = c_cleaned[['customer_id','customer_unique_id','customer_state']]
orders_rfm=orders_rfm.merge(customers_clean,on='customer_id',how='left')

In [36]:
orders_rfm.head()

,order_id,customer_id,order_purchase_timestamp,payment_value,customer_unique_id,customer_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,38.71,7c396fd4830fd04220f754e42b4e5bff,SP
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,2018-07-24 20:41:37,141.46,af07308b275d755c9edb36a90c618231,BA
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,2018-08-08 08:38:49,179.12,3a653a41f6f9fc3d2a113cf8398680e8,GO
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,2017-11-18 19:28:06,72.20,7c142cf63193a1473d2e66489a9ae977,RN
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,2018-02-13 21:18:39,28.62,72632f0f9dd73dfee390c9b22eb56dd6,SP


In [37]:
orders_rfm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96477 entries, 0 to 96476
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   order_id                  96477 non-null  object        
 1   customer_id               96477 non-null  object        
 2   order_purchase_timestamp  96477 non-null  datetime64[ns]
 3   payment_value             96477 non-null  float64       
 4   customer_unique_id        96477 non-null  object        
 5   customer_state            96477 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 4.4+ MB


## Items

In [39]:
items = pd.read_csv(RAW_DIR/"olist_order_items_dataset.csv")

In [40]:
items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [41]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


In [43]:
items.isnull().sum()

order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

In [44]:
items_agg = items.groupby('order_id', as_index=False).agg({
    'price': 'sum',
    'freight_value': 'sum'
})
items_agg.rename(columns={'price': 'total_price', 'freight_value': 'total_freight'}, inplace=True)


In [45]:
items_agg.head()

,order_id,total_price,total_freight
0,00010242fe8c5a6d1ba2dd792cb16214,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,199.90,18.14


## Merging Items and orders_rfm

In [46]:
orders_rfm=orders_rfm.merge(items_agg,on='order_id',how='left')

In [48]:
orders_rfm.head()

,order_id,customer_id,order_purchase_timestamp,payment_value,customer_unique_id,customer_state,total_price,total_freight
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,38.71,7c396fd4830fd04220f754e42b4e5bff,SP,29.99,8.72
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,2018-07-24 20:41:37,141.46,af07308b275d755c9edb36a90c618231,BA,118.70,22.76
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,2018-08-08 08:38:49,179.12,3a653a41f6f9fc3d2a113cf8398680e8,GO,159.90,19.22
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,2017-11-18 19:28:06,72.20,7c142cf63193a1473d2e66489a9ae977,RN,45.00,27.20
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,2018-02-13 21:18:39,28.62,72632f0f9dd73dfee390c9b22eb56dd6,SP,19.90,8.72


In [49]:
orders_rfm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96477 entries, 0 to 96476
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   order_id                  96477 non-null  object        
 1   customer_id               96477 non-null  object        
 2   order_purchase_timestamp  96477 non-null  datetime64[ns]
 3   payment_value             96477 non-null  float64       
 4   customer_unique_id        96477 non-null  object        
 5   customer_state            96477 non-null  object        
 6   total_price               96477 non-null  float64       
 7   total_freight             96477 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 5.9+ MB


In [50]:
orders_rfm['customer_state'] = orders_rfm['customer_state'].astype('category')

In [51]:
orders_rfm = orders_rfm.astype({'order_id': 'string', 'customer_id': 'string', 'customer_unique_id': 'string'})

In [52]:
orders_rfm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96477 entries, 0 to 96476
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   order_id                  96477 non-null  string        
 1   customer_id               96477 non-null  string        
 2   order_purchase_timestamp  96477 non-null  datetime64[ns]
 3   payment_value             96477 non-null  float64       
 4   customer_unique_id        96477 non-null  string        
 5   customer_state            96477 non-null  category      
 6   total_price               96477 non-null  float64       
 7   total_freight             96477 non-null  float64       
dtypes: category(1), datetime64[ns](1), float64(3), string(3)
memory usage: 5.2 MB


In [53]:
orders_rfm.head()

,order_id,customer_id,order_purchase_timestamp,payment_value,customer_unique_id,customer_state,total_price,total_freight
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,38.71,7c396fd4830fd04220f754e42b4e5bff,SP,29.99,8.72
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,2018-07-24 20:41:37,141.46,af07308b275d755c9edb36a90c618231,BA,118.70,22.76
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,2018-08-08 08:38:49,179.12,3a653a41f6f9fc3d2a113cf8398680e8,GO,159.90,19.22
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,2017-11-18 19:28:06,72.20,7c142cf63193a1473d2e66489a9ae977,RN,45.00,27.20
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,2018-02-13 21:18:39,28.62,72632f0f9dd73dfee390c9b22eb56dd6,SP,19.90,8.72


## Saving orders_rfm

In [57]:
orders_rfm.to_csv(PROCESSED_DIR/"orders_rfm.csv", index=False)